# USIDOF : Outil de données - Projet Traiter des fichiers Excel avec avec Python

## Etude et support de travail
Ce projet se porte sur les ventes de véhicules électriques depuis 2011 à 2035 à l'aide de projections basées sur les engagements des pays et des régions à l'instar de l'Europe.Pour étudier ce sujet, j'ai importé des données fournis par l'International Energy Agency (IEA) ainsi que des données géographiques sur Kaggle listant pour chaque pays, le nom de son continent et si oui ou non il fait parti de l'ONU.
Le projet vise à la création d'un tableau de bord dynamique à destination de professionnels de l'automobile.
Toutefois il doit permettre à des personnes non initiés de comprendre le sujet.

## Analyse

Les tableaux de bord analysent pour chaque année entre autres : 
- les ventes de véhicules des régions (EV sales)
- la circulation de véhicules électriques dans ces régions. (EV stock)
- le rapport entre les ventes et la circulation des véhicules
- La demande d'électricité des régions
- l'utilisation de pétrole entre autres


## Méthodologie

Nous avons choisi pour ce projet d'utiliser la librairie Openpyxl pour la création et le travail sur des fichiers excels. Les avantages de cette librairie sont de supporter de nombreuses fonctionnalités d'excel dont la mise en forme, la création de feuilles ainsi que l'utilisation de nombreuses formules, comme on peut le voir dans les bibliothèques téléchargées ci-dessous.
Par ailleurs les sources de documentation d'Openpyxl ont été de façon exhaustive : 
- Documentation officielle de la librairie : https://openpyxl.readthedocs.io/en/stable/
- Tutoriel d'un blogueur ?? : https://realpython.com/openpyxl-excel-spreadsheets-python/
- GeeksforGeeks : https://www.geeksforgeeks.org/python-plotting-charts-in-excel-sheet-using-openpyxl-module-set-2/
- un blog japonais : https://www.shibutan-bloomers.com/python_libraly_openpyxl-11_en/5845/

In [1]:
# Chargement des librairies
import csv
import openpyxl
from openpyxl import Workbook # créer un fichier excel sur Python
from openpyxl import load_workbook # charger un fichier excel 
from openpyxl.utils import FORMULAE
from openpyxl.chart import BarChart, Series, PieChart, LineChart, BubbleChart, Reference # créer des graphiques
from openpyxl.formula.translate import Translator #copie les formules
from openpyxl.utils.dataframe import dataframe_to_rows # créer un dataframe à partir des données d'une feuille excel
from openpyxl.worksheet.table import Table
from openpyxl.utils import get_column_interval
from openpyxl.styles import NamedStyle, Font, PatternFill, Side, Border, Alignment
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.chart.label import DataLabelList
from openpyxl.utils import quote_sheetname
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.chart.marker import Marker
from openpyxl.drawing.fill import PatternFillProperties
from openpyxl.drawing.line import LineProperties
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.drawing.colors import ColorChoice

import pandas as pd
from pandas import DataFrame

from IPython.display import Image

In [2]:
openpyxl.__version__ # vérifier que la version d'OPENPXL supérieur ou égale à la version 3.

'3.1.3'

## Chargement des fichiers csv et conversion en fichier excel
1) fichier csv avec les parts de ventes historiques répertoriés par l'IEA
2) fichier csv avec les parts de ventes projetées par l'IEA
3) fichier csv avec liste des pays et des continents en 2024 source : https://worldpopulationreview.com/country-rankings/list-of-countries-by-continent

In [3]:
wb = openpyxl.Workbook()
filename= 'ev_sales.xlsx'
ws1 = wb.active
ws1.title = "raw_data"
with open('ev_historical.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        ws1.append(row)

wb.save(str(filename))

In [4]:
# charger le second csv et ajouter les observations sur la même feuille 
with open('ev_projection.csv') as f:
    reader = csv.reader(f, delimiter=',')
    next(f)
    for row in reader:
        ws1.append(row)
        #print (row) étape de vérification

wb.save(str(filename))

In [5]:
ws2 = wb.create_sheet("continents",1)
ws2.title = "geo_data"
with open('continent.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        ws2.append(row)
        #print (row) #étape de vérification
wb.save(str(filename))

### Créer un dataframe avec Pandas

Nous crééons un dataframe afin d'explorer et de nettoyer la donnée

In [6]:
data = ws1.values
columns = next(data)  # Première ligne pour récupérer le nom des variables

# création du dataframe sur lequel on va travailler 
df = pd.DataFrame(data, columns=columns)
df.rename(columns={"region": "country"},inplace=True)
# Supprimer les lignes contenant des valeurs nulles dans la première colonne
df = df.dropna(subset=[df.columns[0]])

# Etape de vérification du dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4249 entries, 0 to 4248
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     4249 non-null   object
 1   category    4249 non-null   object
 2   parameter   4249 non-null   object
 3   mode        4249 non-null   object
 4   powertrain  4249 non-null   object
 5   year        4249 non-null   object
 6   unit        4249 non-null   object
 7   value       4249 non-null   object
dtypes: object(8)
memory usage: 298.8+ KB


In [7]:
df.head()

,country,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.006500000134110451
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039000000106170774
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220


In [8]:
data_c = ws2.values
columns_c = next(data_c)  # Première ligne pour récupérer le nom des variables

# création du dataframe sur lequel on va travailler 
df_c = pd.DataFrame(data_c, columns=columns_c)

# Supprimer les lignes contenant des valeurs nulles dans la première colonne
df_c = df_c.dropna(subset=[df_c.columns[0]])

# Etape de vérification du dataframe
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 0 to 233
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    234 non-null    object
 1   continent  234 non-null    object
 2   unMember   234 non-null    object
dtypes: object(3)
memory usage: 7.3+ KB


In [9]:
# Etape de vérification 
df_c.head()

,country,continent,unMember
0,India,Asia,true
1,China,Asia,true
2,United States,North America,true
3,Indonesia,Asia,true
4,Pakistan,Asia,true


#### Définir les dimensions du dataframe ev_sales

In [10]:
column_a = ws1['A']
max_row = len(column_a)
min_row = 2

#### Corriger les erreurs du dataframe colonne pays avec pandas car problème avec la formule excel

In [11]:
def unique_values(min_col, max_col, ws):
    unique_value = []
    for row in ws.iter_rows(min_col =min_col, max_col=max_col):
        for cell in row:
            unique_value.append(cell.value)
    unique_values = list(set(unique_value))
    print(unique_values)
unique_values(1,1,ws=ws1)

['Turkiye', 'Latvia', 'Costa Rica', 'Romania', 'EU27', 'Ireland', 'Netherlands', 'Sweden', 'Rest of the world', 'Switzerland', 'Colombia', 'China', 'Austria', 'Germany', 'Slovenia', 'Norway', 'Japan', 'Lithuania', 'United Kingdom', 'Greece', 'Seychelles', 'Portugal', 'Spain', 'Estonia', 'Italy', 'Europe', 'Denmark', 'Croatia', 'Bulgaria', 'Belgium', 'Brazil', 'Finland', 'Korea', 'Australia', 'Luxembourg', 'South Africa', 'Cyprus', 'Chile', 'New Zealand', 'Iceland', 'India', 'region', 'Israel', 'Slovakia', 'World', 'Canada', 'France', 'United Arab Emirates', 'Czech Republic', 'Mexico', 'USA', 'Poland', 'Hungary']


In [12]:
df['country'] = df['country'].replace('Turkiye','Turkey')
df['country'] = df['country'].replace('USA','United States')
df['country'] = df['country'].replace('Korea','South Korea')
# Etape de vérification
df[df.eq("Turkey").any(1)]

,country,category,parameter,mode,powertrain,year,unit,value
3318,Turkey,Historical,EV sales share,Cars,EV,2012,percent,0.01600000075995922
3319,Turkey,Historical,EV sales,Cars,BEV,2012,Vehicles,92
3320,Turkey,Historical,EV sales,Cars,PHEV,2013,Vehicles,1
3321,Turkey,Historical,EV stock,Cars,BEV,2013,Vehicles,230
3322,Turkey,Historical,EV sales share,Cars,EV,2013,percent,0.00011999999696854502
...,...,...,...,...,...,...,...,...
3377,Turkey,Historical,EV stock,Cars,PHEV,2023,Vehicles,4900
3378,Turkey,Historical,EV stock,Cars,BEV,2023,Vehicles,77000
3379,Turkey,Historical,EV sales,Cars,BEV,2023,Vehicles,66000
3380,Turkey,Historical,EV sales share,Cars,EV,2023,percent,10


In [13]:
# Modifier le type de données de certaines variables
df['year'] = pd.to_datetime(df['year'], errors='coerce').dt.year # définit une date courte format YYYY
df['value'] = df['value'].astype(float)
df['category'] = df['category'].astype('category')
print(df.info()) #étape de vérification

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4249 entries, 0 to 4248
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   country     4249 non-null   object  
 1   category    4249 non-null   category
 2   parameter   4249 non-null   object  
 3   mode        4249 non-null   object  
 4   powertrain  4249 non-null   object  
 5   year        4249 non-null   int64   
 6   unit        4249 non-null   object  
 7   value       4249 non-null   float64 
dtypes: category(1), float64(1), int64(1), object(5)
memory usage: 269.8+ KB
None


In [14]:
print(df.head())

     country    category       parameter  mode powertrain  year      unit  \
0  Australia  Historical  EV sales share  Cars         EV  2011   percent   
1  Australia  Historical  EV stock share  Cars         EV  2011   percent   
2  Australia  Historical        EV sales  Cars        BEV  2011  Vehicles   
3  Australia  Historical        EV stock  Cars        BEV  2011  Vehicles   
4  Australia  Historical        EV stock  Cars        BEV  2012  Vehicles   

       value  
0    0.00650  
1    0.00039  
2   49.00000  
3   49.00000  
4  220.00000  


### Copier le dataframe dans un fichier excel

In [15]:
# Création d'une feuille excel
ws3 = wb.create_sheet("ev_sales_df",2)
wb.sheetnames

['raw_data', 'geo_data', 'ev_sales_df']

In [16]:
### Etape de vérification du nom des colonnes
df.columns
df_c.columns

Index(['country', 'continent', 'unMember'], dtype='object')

In [17]:
# Copier le df dans la feuille excel ws3 nommé ev_sales_df
rows = dataframe_to_rows(df, index=False)
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
        ws3.cell(row=r_idx, column=c_idx, value=value)
wb.save(str(filename))

In [18]:
### Vérification de ce que contient la feuille de travail excel
row_check = ws3['2']
for cell in row_check:
    print(cell.value)

Australia
Historical
EV sales share
Cars
EV
2011
percent
0.006500000134110451


### Nettoyer et modifier la data sur excel avec des formules 

###### Cette formule n'a pas été utilisée car elle ne permettait pas ensuite de copier les valeurs uniques car les valeurs restent des formules et non des valeurs brutes
###### Objectif de cette formule avec SUBSTITUTE : remplacer les valeurs pays mal ortographiés : Korea, USA, Turkiye

###### for r in ws3.iter_rows(min_col=1, max_col=1):
######    for cell in r:
######        if cell.row>1:
######            cell.value= '=SUBSTITUTE(SUBSTITUTE(SUBSTITUTE(B' + str(cell.row) + ',"Turkiye","Turkey"),"USA","United States"),"Korea","South Korea")'
######        print((cell.value))


#### Ajouter une colonne continent avec RECHERCHEX()/XLOOKUP
Cette formule recherche le continent correspondant à chaque pays du dataframe.
Certaines formules ne font pas parti de la librairie openpyxl, il faut précéder les formules excel de "_xlfn".

Aussi pour incrémenter une formule dans plusieurs cellules, dans une colonne notamment, on définit uen cellule de départ, on créé une boucle dans laquelle on définit les cellules dans lesquelles la formule doit être incrémentée. Puis on inclut dans la formule la variable {row} qui appartient au range des cellules à incrémenter.

In [19]:
continent = ws3.insert_cols(3) # insertion colonne C
ws3.cell(row=1, column=3, value="continent") 
for row in range(2, ws3.max_row +1):
    ws3[f'C{row}'] = f'=_xlfn.XLOOKUP(A{row},geo_data!A:A,geo_data!B:B,"",0,1)'
    #print((cell.value))
wb.save(str(filename))

In [20]:
# vérification
row_check = ws3['1']
for cell in row_check:
    print(cell.value)

country
category
continent
parameter
mode
powertrain
year
unit
value


In [21]:
# vérifier la dimension de notre plage de données 
ws3.calculate_dimension()

'A1:I4250'

In [22]:
# Vérification du résultat
row_check = ws3['2']
for cell in row_check:
    print(cell.value)

Australia
Historical
=_xlfn.XLOOKUP(A2,geo_data!A:A,geo_data!B:B,"",0,1)
EV sales share
Cars
EV
2011
percent
0.006500000134110451


#### Ajouter une colonne Année avec la formule GAUCHE()/LEFT()
Cette formule retourne les 4 premiers caractères en partant de la gauche d'une cellule précise.
L'objectif était de retourner l'année en format numérique si besoin toutefois après vérification je n'ai pas pu obtenir que la valeur des cellules mais les formules donc la réutilisation n'était pas possible. 

C'est pourquoi, j'ai modifié le nettoyage Pandas pour contrecarrer ce problème en modifiant la variable year avec ce code vu ci-dessus : "df['year'] = pd.to_datetime(df['year'], errors='coerce').dt.year"

In [23]:
# Insérer une colonne à la position 8 (colonne I)
ws3.insert_cols(8)

# Définir le titre de la nouvelle colonne
ws3.cell(row=1, column=8, value="year_corr")

# Remplir les cellules de la nouvelle colonne avec la formule
for r in range(2, ws3.max_row + 1): # Commence à la ligne 2 pour ignorer le titre
    cell = ws3.cell(row=r, column=8)
    cell.value = f'=LEFT(G{r}, 4)'

# Sauvegarder le classeur
wb.save(str(filename))

In [24]:
# vérifier la dimension de notre plage de données 
ws3.calculate_dimension()

'A1:J4250'

### Page de paramètres
La page de paramètres contient les valeurs uniques de chaque variable afin de créer des listes de validation.

In [25]:
# Vérifier si la feuille existe sinon la créé
if 'Parameters' not in wb.sheetnames:
    # Créer la feuille
    ws5 = wb.create_sheet('parameters')
else :
    ws5 = wb['parameters']

#### Continent

In [26]:
from openpyxl import Workbook, load_workbook

def copier_valeurs_uniques(wb, ws_source, ws_target, col_start, row_start, col_target):    
    values = []
    for row in ws_source.iter_rows(min_col=col_start, max_col=col_start, min_row=row_start):
        for cell in row:
            values.append(cell.value)
    
    unique_values = list(set(values))
    print(unique_values)  # étape de vérification

    for i, value in enumerate(sorted(unique_values), start=1):
        ws_target.cell(row=i, column=col_target, value=value)
    
    wb.save(str(filename))
    
# continent
copier_valeurs_uniques(wb, ws_source= ws2, ws_target= ws5, col_start=2, row_start=2, col_target=1)

['Asia, Europe', 'Asia', 'Africa', 'North America', 'South America', 'Oceania', 'Europe']


In [27]:
# country
copier_valeurs_uniques(wb, ws_source= ws3, ws_target= ws5, col_start=1, row_start=2, col_target=2)

['Latvia', 'Costa Rica', 'Romania', 'EU27', 'Ireland', 'Netherlands', 'Sweden', 'Rest of the world', 'Switzerland', 'Colombia', 'China', 'Turkey', 'Austria', 'Germany', 'Slovenia', 'Norway', 'Japan', 'Lithuania', 'United Kingdom', 'Greece', 'Seychelles', 'Portugal', 'Spain', 'Estonia', 'Italy', 'Europe', 'Denmark', 'Croatia', 'Bulgaria', 'Belgium', 'Brazil', 'Finland', 'Australia', 'Luxembourg', 'South Africa', 'Cyprus', 'Chile', 'New Zealand', 'Iceland', 'United States', 'India', 'Israel', 'Slovakia', 'World', 'Canada', 'France', 'South Korea', 'United Arab Emirates', 'Czech Republic', 'Mexico', 'Poland', 'Hungary']


In [28]:
# Timeframe data
copier_valeurs_uniques(wb, ws_source= ws3, ws_target= ws5, col_start=7, row_start=2, col_target=3)

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2025, 2030, 2035, 2010, 2011, 2012, 2013, 2014, 2015]


In [29]:
# category
copier_valeurs_uniques(wb, ws_source= ws3, ws_target= ws5, col_start=2, row_start=2, col_target=4)

['Projection-STEPS', 'Historical']


In [30]:
# parameter
copier_valeurs_uniques(wb, ws_source= ws3, ws_target= ws5, col_start=4, row_start=2, col_target=5)

['EV sales share', 'EV stock', 'EV sales', 'EV stock share', 'Electricity demand', 'Oil displacement Mbd', 'Oil displacement, million lge']


In [31]:
# powertrain
copier_valeurs_uniques(wb, ws_source= ws3, ws_target= ws5, col_start=6, row_start=2, col_target=6)

['PHEV', 'EV', 'BEV', 'FCEV']


In [32]:
# unit
copier_valeurs_uniques(wb, ws_source= ws3, ws_target= ws5, col_start=9, row_start=2, col_target=7)

['Vehicles', 'GWh', 'Oil displacement, million lge', 'percent', 'Milion barrels per day']


### Créer des KPI

Je créé une page avec des KPI sous forme de plusieurs tableaux de bord avec des valeurs en lien avec un seul sujet car le jeu de données est varié. 
Cette feuille sera l'arrière plan de la construction des graphiques qui seront sur une autre page : le dashboard (tableau de bord).

#### Créer des listes de validations

In [33]:
# créer les feuilles KPI et Dashboard
ws6 = wb.create_sheet("KPI")
ws6 = wb["KPI"]
ws7 = wb.create_sheet("Dashboard")
ws7 = wb["Dashboard"]


# première liste de validation
dv1 = DataValidation(type="list", formula1="=parameters!$B$1:$B$53",allow_blank=True, showDropDown=False)
dv1.error ='Your entry is not in the list'
dv1.errorTitle ='Invalid entry'
dv1.prompt = 'Please select from the list'
dv1.promptTitle = 'List Selection'

# deuxième liste de validation
dv2 = DataValidation(type="list", formula1="=parameters!$C$1:$C$18", allow_blank=True, showDropDown=False)
dv2.error ='Your entry is not in the list'
dv2.errorTitle ='Invalid entry'
dv2.prompt = 'Please select from the list'
dv2.promptTitle = 'List Selection'


wb.save(str(filename))

In [34]:
# Ajouter des listes de validation sur le dashboard
# Application de la validation des données à une plage de cellules
ws7.add_data_validation(dv1)
d11 = ws7["D11"]
ws7.column_dimensions['D'].width = 20
d11.value = "Select country"
dv1.add(d11)

ws7.add_data_validation(dv2)
d34 = ws7["D34"]
d34.value = "Select Year"
dv2.add(d34)

wb.save(str(filename))

#### Ajouter des tableaux de bords sur la feuille KPI 

In [35]:
# Créer plusieurs styles à appliquer aux KPI et au dashboard
Style1 = openpyxl.styles.NamedStyle(name = 'Style1')
Style1.font = openpyxl.styles.Font(name = 'Arial', size = 11, color = 'FFFFFF')
Style1.fill = openpyxl.styles.PatternFill(patternType = 'solid', fgColor = '002060')
borderStyle = openpyxl.styles.Side(style = 'medium', color = '002060')
Style1.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True) 
Style1.border = openpyxl.styles.Border(left = borderStyle, right = borderStyle, top = borderStyle, bottom = borderStyle)

Style2 = openpyxl.styles.NamedStyle(name = 'Style2')
Style2.font = openpyxl.styles.Font(name = 'Arial', size = 18, color = 'FF000000', bold=True)
Style2.fill = openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'FFD700')
borderStyle = openpyxl.styles.Side(style = 'medium', color = 'FFD700')
Style2.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True) 
Style2.border = openpyxl.styles.Border(left = borderStyle, right = borderStyle, top = borderStyle, bottom = borderStyle)

Style3 = openpyxl.styles.NamedStyle(name = 'Style3')
Style3.font = openpyxl.styles.Font(name = 'Arial', size = 18, color = 'FF000000', underline='single', bold=True)
Style3.fill = openpyxl.styles.PatternFill(patternType = 'solid', fgColor = 'FFFFFF')
borderStyle = openpyxl.styles.Side(style = 'medium', color = 'FFFFFF')
Style3.border = openpyxl.styles.Border(left = borderStyle, right = borderStyle, top = borderStyle, bottom = borderStyle)


Style4 = openpyxl.styles.NamedStyle(name='Style4')
Style4.font = openpyxl.styles.Font(name = 'Calibri', size = 36, color = 'FFFFFF', bold=True)
Style4.fill = openpyxl.styles.PatternFill(patternType = 'solid', fgColor = '002060')
borderStyle = openpyxl.styles.Side(style = 'medium', color = '002060')
Style4.border = openpyxl.styles.Border(left = borderStyle, right = borderStyle, top = borderStyle, bottom = borderStyle)
Style4.font.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)  
double_border_side = Side(border_style="double")


# Ajout des styles au classeur
wb.add_named_style(Style1)
wb.add_named_style(Style2)
wb.add_named_style(Style3)
wb.add_named_style(Style4)

In [36]:
# créer une barre à gauche des KPI et du dashboard 
ws6 = wb["KPI"]
a1 = ws6["A1"]
navigation = ws6.merge_cells('A1:A100')
a1.style = Style1

In [37]:
ws7 = wb["Dashboard"]
# affecter un style aux listes de validations
d11.style= Style2
d34.style = Style2

#### Titre des différents KPI

In [38]:
#kpi n°1
ws6 = wb["KPI"]
c5 = ws6["C5"]
c5.value = "KPI n°1 : Car Sales by country"
c5.style = Style3
# A EFFACER ws6.merge_cells('C5:E5')

#kpi n°2
g5 = ws6["G5"]
g5.value = "KPI n°2: Global repartition of car powertrain in sales"
g5.style = Style3
# A EFFACER ws6.merge_cells('G5:L5')
 
#kpi n°3
# A EFFACER ws6.merge_cells('C16:F16')
c16 = ws6["C16"]
c16.value = "KPI n°3: Continent EV stock since 2011"
c16.style = Style3

# kpi n°4
# A EFFACER ws6.merge_cells('G16:L16')
g16 = ws6["G16"]
g16.value = "KPI n°4: Global evolution of BEV and PHEV sales"
g16.style = Style3

# kpi n°5
#A EFFACER  ws6.merge_cells('N5:S5')
c30 = ws6["N5"]
c30.value = "KPI n°5: Biggest EV saler"
c30.style = Style3

# kpi n°6
# A EFFACER ws6.merge_cells('C29:E29')
c30 = ws6["C29"]
c30.value = "KPI n°6: Oil displacement and Electricity demand"
c30.style = Style3

Pour créer les KPIS suivant, j'utilise exclusivement des formules excel. 
Pour information, elles doivent être en anglais et les séparations sont des virgules et non des points-virgules. Aussi, les côtes au sein de la formule et celles qui entourent la formule doivent être différentes.

De plus, il faut absolument prêter attention aux cellules auxquelles on applique la formule et celles qu'on appelle dans la formule. Les fonctions permettent d'alléger ce travail de repérage.

#### KPI n°1 : Car sales by country - historical/projection/evolution


In [39]:
# première table KPI
ws6["C9"].value = "Historical"
ws6["D9"].value = "Projection"
ws6["E9"].value = "Evolution"
ws6["C9"].style = Style1
ws6["D9"].style = Style1
ws6["E9"].style = Style1

wb.save(str(filename))

In [40]:
# Calcul KPI projection/historical
def kpi1(ws,start_col, start_row,category):
    for r in ws.iter_rows(min_col=start_col, max_col=start_col, min_row=start_row, max_row=start_row):
        for cell in r:
            if cell.row>=1:
                cell.value= f'=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,Dashboard!D11,ev_sales_df!$B$2:$B$4250,{category},ev_sales_df!$I$2:$I$4250,"Vehicles")'
            print((cell.value))
    wb.save(str(filename))

In [41]:
kpi1(ws6,3,10,'"Historical"')

=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,Dashboard!D11,ev_sales_df!$B$2:$B$4250,"Historical",ev_sales_df!$I$2:$I$4250,"Vehicles")


In [42]:
kpi1(ws6,4,10,'"Projection-STEPS"')

=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,Dashboard!D11,ev_sales_df!$B$2:$B$4250,"Projection-STEPS",ev_sales_df!$I$2:$I$4250,"Vehicles")


In [43]:
# Evolution value
for r in ws6.iter_rows(min_col=5, max_col=5, min_row=10, max_row=10):
    for cell in r:
        if cell.row>=1:
            cell.value= '=IFERROR(ROUND(C10/D10*100,0)& "%","no projection data")'
        print((cell.value))
wb.save(str(filename))

=IFERROR(ROUND(C10/D10*100,0)& "%","no projection data")


#### KPI n°2: Global repartition of car powertrain in sales

Ce KPI enregistre pour une année donnée dans une liste de validation, la répartition des moteurs à propulsion des voitures vendues dans le monde (variable Country "World")

In [44]:
# deuxième table KPI
ws6["G9"].value = "Powertrain"
ws6["H9"].value = "Vehicle sales"
ws6["I9"].value = "EV total sales share"
ws6["G10"].value = "BEV"
ws6["G11"].value = "PHEV"
ws6["G12"].value = "FCEV"
ws6["G13"].value = "Others incl. diesel and petrol (estim)"
ws6["G14"].value = "Total estimated"

ws6["G9"].style = Style1
ws6["H9"].style = Style1
ws6["I9"].style = Style1
ws6["G10"].style = Style1
ws6["G11"].style = Style1
ws6["G12"].style = Style1
ws6["G13"].style = Style1
ws6["G14"].style = Style1

wb.save(str(filename))

In [45]:
# EV sales shares
for r in ws6.iter_rows(min_col=9, max_col=9, min_row=10, max_row=10):
    for cell in r:
        if cell.row>=1:
            cell.value= '=ROUND(SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales share",ev_sales_df!$A$2:$A$4250,"World")/COUNTIFS(ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales share",ev_sales_df!$I$2:$I$4250,"percent",ev_sales_df!$A$2:$A$4250,"World"),2)&"%"'
        print((cell.value)) # étape de vérification
wb.save(str(filename))

=ROUND(SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales share",ev_sales_df!$A$2:$A$4250,"World")/COUNTIFS(ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales share",ev_sales_df!$I$2:$I$4250,"percent",ev_sales_df!$A$2:$A$4250,"World"),2)&"%"


In [46]:
def kpi2(ws,start_col, start_row,powertrain):
    for r in ws.iter_rows(min_col=start_col, max_col=start_col, min_row=start_row, max_row=start_row):
        for cell in r:
            if cell.row>=1:
                cell.value= f'=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales",ev_sales_df!$A$2:$A$4250,"World",ev_sales_df!$F$2:$F$4250,{powertrain})'
            print((cell.value))
    wb.save(str(filename))

In [47]:
kpi2(ws6,8,10,'"BEV"')
kpi2(ws6,8,11,'"PHEV"')
kpi2(ws6,8,12,'"FCEV"')

=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales",ev_sales_df!$A$2:$A$4250,"World",ev_sales_df!$F$2:$F$4250,"BEV")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales",ev_sales_df!$A$2:$A$4250,"World",ev_sales_df!$F$2:$F$4250,"PHEV")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$G$2:$G$4250,Dashboard!D34,ev_sales_df!$D$2:$D$4250,"EV sales",ev_sales_df!$A$2:$A$4250,"World",ev_sales_df!$F$2:$F$4250,"FCEV")


In [48]:
# Estimated total number of vehicles
for r in ws6.iter_rows(min_col=8, max_col=8, min_row=14, max_row=14):
    for cell in r:
        if cell.row>=1:
            cell.value= '=ROUND((SUM(KPI!$H$10:$H$12)*100%)/$I$10,0)'
        print((cell.value))
wb.save(str(filename))

=ROUND((SUM(KPI!$H$10:$H$12)*100%)/$I$10,0)


In [49]:
# Estimated number of others including diesel and petrol
for r in ws6.iter_rows(min_col=8, max_col=8, min_row=13, max_row=13):
    for cell in r:
        if cell.row>=1:
            cell.value= '=ROUND($H$14-SUM($H$10:$H$12),0)'
        #print((cell.value))
wb.save(str(filename))

### KPI n°3: Historical EV stock by continent

Ce KPI relève le nombre voitures électriques en circulation dans les continents à une année donnée et compare cette valeur à la part des voitures électriques au sein des continent en 2023 (EV stock share).

In [50]:
# 3ième table KPI
ws6["C18"].value = "Continent"
ws6["D18"].value = "EV Stock"
ws6["E18"].value = "EV Stock share in 2023"

ws6["C18"].style = Style1
ws6["D18"].style = Style1
ws6["E18"].style = Style1
# A EFFACER ws6.column_dimensions['E'].width = 35
# A EFFACER ws6.column_dimensions['L'].width = 35

wb.save(str(filename))

In [51]:
ws_w5 = wb['parameters']
ws_w6 = wb['KPI']

# Copier les cellules de A1:A8 de w5 vers C19:C25 de w6
for i in range(7):
    value = ws_w5[f'A{i + 1}'].value
    ws_w6[f'C{i + 19}'].value = value 
wb.save(str(filename))

In [52]:
# EV stock

for r in ws6.iter_rows(min_col=4, max_col=4, min_row=19, max_row=25):
    for cell in r:
        if cell.row>=1:
            cell.value= f'=ROUND(SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$D$2:$D$4250,$D$18,ev_sales_df!$C$2:$C$4250,$C{cell.row},ev_sales_df!$B$2:$B$4250,"Historical"),0)'
        #print((cell.value))
wb.save(str(filename))

In [53]:
# EV stock share
for r in ws6.iter_rows(min_col=5, max_col=5, min_row=19, max_row=25):
    for cell in r:
        if cell.row>=1:
            cell.value= f'=ROUND(AVERAGEIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$D$2:$D$4250,"EV stock share",ev_sales_df!$C$2:$C$4250,$C{cell.row},ev_sales_df!$G$2:$G$4250,"2023"),2)'
        #print((cell.value))
wb.save(str(filename))

### KPI n°4 : Evolution of BEV and PHEV

Ce KPI note l'évolution de la vente de voitures électriques (BEV et PHEV) les plus "vertes" d'année en année de 2010 à 2023. Je n'ai pas réussi à créer une visualisation avec un graphique en cascade pour montrer l'évolution en pourcentage d'année en année, mais je l'ai calculé dans le KPI dans la page "parameters".

In [54]:
# 4ième table KPI
ws6["G18"].value = "Year"
ws6["H18"].value = "# of BEV sales World"
ws6["I18"].value = "# of PHEV sales World"
ws6["J18"].value = "Evolution BEV"
ws6["K18"].value = "Evolution PHEV"

ws6["G18"].style = Style1
ws6["H18"].style = Style1
ws6["I18"].style = Style1
ws6["J18"].style = Style1
ws6["K18"].style = Style1

wb.save(str(filename))

In [55]:
ws_w5 = wb['parameters']
ws_w6 = wb['KPI']

# Copier les cellules de C1:C17 de w5 vers G19:G35 de w6
for i in range(17):
    value = ws_w5[f'C{i + 1}'].value
    ws_w6[f'G{i + 19}'].value = value 
wb.save(str(filename))

In [56]:
def kpi4(ws,start_col, start_row, max_row, powertrain):
    for r in ws.iter_rows(min_col=start_col, max_col=start_col, min_row=start_row, max_row=max_row):
        for cell in r:
            if cell.row>=1:
                cell.value= f'=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$D$2:$D$4250,"EV sales",ev_sales_df!$F$2:$F$4250,{powertrain},ev_sales_df!$G$2:$G$4250,$G{cell.row},ev_sales_df!$A$2:$A$4250,"World")'
            #print((cell.value)) # Etape de vérification 
    wb.save(str(filename))
    
# Sum of BEV and PHEV
kpi4(ws6,8,19,35,'"BEV"')
kpi4(ws6,9,19,35,'"PHEV"')

In [57]:
def kpi4_evolution(ws, start_col, start_row, end_row):
    for r in ws.iter_rows(min_col=start_col, max_col=start_col, min_row=start_row, max_row=end_row):
        for cell in r:
            if cell.row >= start_row:  # Commencer à la ligne 19 pour éviter la première ligne
                previous_row = cell.row - 1
                cell.value = f'=IFERROR(ROUND(($H{cell.row}-$H{previous_row})/$H{previous_row}*100,0),0)'
                #print(cell.value)  
            else:
                cell.value = 0  # base reference, not possible to divide with a zero
wb.save(str(filename))

# Evolution of BEV and PHEV
kpi4_evolution(ws6,10,19,35)
kpi4_evolution(ws6,11,19,35)

### KPI n°5: Biggest EV saler

Ce KPI relève le pays ou la région (variable "country" qui comprend des régions comme EU27) ayant vendu le plus de véhicules électriques à une année donnée par la liste de validation. L'objectif implicite est de connaître le pays le plus présent dans la transition des flottes automobiles. Toutefois cette donnée seule ne suffit pas à aboutir à une telle interprétation, elle donne une tendance.

In [58]:
# 5ième table KPI A FAIRE
ws6["O9"].value = "Country"
ws6["N9"].value = "EV sales"

ws6["O9"].style = Style1
ws6["N9"].style = Style1

wb.save(str(filename))

In [59]:
ws_w5 = wb['parameters']
ws_w6 = wb['KPI']

# Copier les cellules de B1:C51 de w5 vers O10:O60 de w6
for i in range(51):
    value = ws_w5[f'B{i + 1}'].value
    ws_w6[f'O{i + 10}'].value = value 
wb.save(str(filename))

In [60]:
# of EV sales
for r in ws6.iter_rows(min_col=14, max_col=14, min_row=10, max_row=60):
    for cell in r:
        if cell.row>=10:
            cell.value= f'=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,$O{cell.row},ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$I$2:$I$4250,"Vehicles",ev_sales_df!$D$2:$D$4250,"EV sales")'
        #print(cell.value)  
wb.save(str(filename))

In [61]:
# sort countries with most ev sales - output an AttributeError but it works
list2 = ws7['AR45']
formula = '=_xlfn.SORT(KPI!N10:O60,1,-1,FALSE)'
list2.value= formula

In [62]:
ws6.formula_attributes['AR45'] = {'t': 'array', 'ref': "KPI!N10:O60"} # .ou _

AttributeError: 'Worksheet' object has no attribute 'formula_attributes'

In [63]:
# name of country with most ev sale sorted table of biggest ev saler avec VLOOKUP
for r in ws7.iter_rows(min_col=44, max_col=44, min_row=42, max_row=42):
    for cell in r:
        if cell.row>=42:
            cell.value= '=_xlfn.VLOOKUP($AR$45,KPI!N10:O60,2,0)'
        print(cell.value)
        cell.font = openpyxl.styles.Font(name = 'Arial', size = 14, bold= True, color = 'FFFFFF')
        cell.fill = openpyxl.styles.PatternFill(patternType = 'solid', fgColor = '002060')
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True) 
        cell.border = openpyxl.styles.Border(left = borderStyle, right = borderStyle, top = borderStyle, bottom = borderStyle) 
wb.save(str(filename))

=_xlfn.VLOOKUP($AR$45,KPI!N10:O60,2,0)


### KPI n°6: Oil displacement and Electricity demand

Ce tableau compare pour les 3 plus grandes régions, les valeurs en déplacement de pétrole et en demande d'électricité pour une année donnée.

In [64]:
# 6ième table KPI
ws6["C33"].value = "Region"
ws6["D33"].value = "Oil displacement (million lge)"
ws6["E33"].value = "Electricity demand (GWh)"

ws6["C34"].value = "China"
ws6["C35"].value = "Europe"
ws6["C36"].value = "United States"

ws6["C33"].style = Style1
ws6["D33"].style = Style1
ws6["E33"].style = Style1
ws6["C34"].style = Style1
ws6["C35"].style = Style1
ws6["C36"].style = Style1

wb.save(str(filename))

In [65]:
def kpi6(ws, start_col, start_row, end_row, subject):
    for r in ws.iter_rows(min_col=start_col, max_col=start_col, min_row=start_row, max_row=end_row):
        for cell in r:
            if cell.row>=1:
                cell.value= f'=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C{cell.row},ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,{subject})'
            print(cell.value)
wb.save(str(filename))

## Oil displacement (million lge) avec SUMIFS
kpi6(ws6,4,34,36,'"Oil displacement, million lge"')
# Electricity demand avec SUMIFS
kpi6(ws6,5,34,36,'"Electricity demand"')

=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C34,ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,"Oil displacement, million lge")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C35,ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,"Oil displacement, million lge")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C36,ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,"Oil displacement, million lge")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C34,ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,"Electricity demand")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C35,ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,"Electricity demand")
=SUMIFS(ev_sales_df!$J$2:$J$4250,ev_sales_df!$A$2:$A$4250,C36,ev_sales_df!$G$2:$G$4250,Dashboard!$D$34,ev_sales_df!$D$2:$D$4250,"Electricity demand")


### Complete dashboards with graphs

In [66]:
# Créer une barre de navigation sur le dashboard
navigation = ws7.merge_cells('A1:A100')
a1_d = ws7["A1"]
ws7.column_dimensions['A'].width = 40
a1_d.style = Style1

In [67]:
# Afficher un titre au dashboard
title_dashboard = ws7["C3"]
ws7["C3"].value = "Global EV outlook 2023 by IEA"
ws7["C3"].style = Style4
title_dashboard = ws7.merge_cells('C3:AP6')

In [68]:
# Ajouter le titre du kpi 5
kpi5 = ws7["AR39"]
ws7["AR39"].value = "Region with greater EV sales this year"
kpi5.style = Style3
kpi5 = ws7.merge_cells('AR39:AY39')
ws7["AS45"].value = "sold cars by the country this year"

# Merge cell with region with greater EV sales this year
subtitle = ws7.merge_cells('AR42:AS43')

In [69]:
# ajouter un sous-titre au dashboard
subtitle = ws7.merge_cells('J8:AE9')
subtitle = ws7["J8"]
subtitle.style = Style2
subtitle.value = "Historical and projection data on electric vehicles and global trends"

In [70]:
# Ajouter un bloc d'information
information = ws7["AR17"]
information.value = "INFORMATION"
information.style = Style1
information = ws7.merge_cells('AR17:AU18')

In [71]:
# alimentation du bloc d'information
intro = ws7["AR19"]
intro.value = "EV sales are growing each year and this trend have influence on oil displacement and electricy demand."

# style of cells
center_aligned_text = Alignment(horizontal="center", wrap_text=True)

yellowfill = PatternFill(start_color='FFD700',
                   end_color='FFD700',
                   fill_type='solid')
for cell in ws7['AR19:AR24']:
   cell[0].fill = yellowfill
intro = ws7.merge_cells('AR19:AU24')


ws7["AR19"].alignment = center_aligned_text

In [72]:
# ajout d'informations sur le dashboard
ws7["AR25"].value = "EV"
ws7["AR26"].value = "PHEV"
ws7["AR27"].value = "FCEV"
ws7["AR28"].value = "BEV"
ws7["AR29"].value = "EV stock"
ws7["AS25"].value = "Vehicles working with electricity partially or fully"
ws7.merge_cells('AS25:AU25')
ws7["AS26"].value = "Plug-in Hybrid electric vehicle"
ws7.merge_cells('AS26:AU26')
ws7["AS27"].value = "Fuel cell electric vehicle"
ws7.merge_cells('AS27:AU27')
ws7["AS28"].value = "Batteric Electric Vehicle (100% electric)"
ws7.merge_cells('AS28:AU28')
ws7["AS29"].value = "Electric vehicle on roads"
ws7.merge_cells('AS29:AU29')

# style information on dashboard
yellowfill = PatternFill(start_color='FFD700',
                   end_color='FFD700',
                   fill_type='solid')
for cell in ws7['AR25:AR29']:
   cell[0].fill = yellowfill
for cell in ws7['AS25:AS29']:
    cell[0].fill = yellowfill

In [73]:
# Function to create different graph
def create_chart(chart_type, title, data_reference, category_reference, legend_position, chart_position, ws, shape, x_title=None, y_title=None, width=17, height=10, style=10):
    # Créer un objet de graphique en fonction du type
    if chart_type == "bar":
        chart = BarChart()
        chart.type = "col"  # type colonne pour diagramme à barres
        # Définir les axes uniquement pour les graphiques à barres
        if x_title:
            chart.x_axis.title = x_title
        if y_title:
            chart.y_axis.title = y_title
    elif chart_type.lower() == "pie":
        chart = PieChart()
    elif chart_type.lower() == "bubble":
        chart = BubbleChart()
        if x_title:
            chart.x_axis.title = x_title
        if y_title:
            chart.y_axis.title = y_title
    else:
        raise ValueError("Type de graphique non supporté")
    
    # Définir les titres
    chart.title = title

    # Ajouter les données et les catégories
    chart.add_data(data_reference, titles_from_data=True)
    chart.set_categories(category_reference)

    # Personnaliser la légende et la taille
    chart.legend.position = legend_position
    chart.width = width
    chart.height = height
    chart.style = style
    chart.shape = shape

    # Positionner le graphique dans la feuille Excel
    ws.add_chart(chart, chart_position)
    wb.save(str(filename))


In [74]:
# kpi n°1 EV historical sales by country 
ws7 = wb["Dashboard"]
categories_bar = Reference(ws6, min_col=3, min_row=7, max_row=7)  # Catégories pour l'axe des x
data_bar = Reference(ws6, min_col=3, min_row=9, max_col=4, max_row=10)  # Données

create_chart(
    chart_type="bar", 
    title="Car sales", 
    x_title="Sales", 
    y_title="", 
    data_reference=data_bar, 
    category_reference=categories_bar, 
    shape = 5,
    legend_position="b", 
    chart_position="D13", 
    ws=ws7, 
    width=17, 
    height=10,
    style=10
    
)

In [75]:
# Pie chart kpi n°2 repartition car sales powertrain in the world

categories_pie = Reference(ws6, min_col = 7, min_row = 10, max_row = 13)  # Catégories pour l'axe des x
data_pie = Reference(ws6, min_col = 8, max_col = 8, min_row = 9, max_row = 13)  # Données

create_chart(
    chart_type="pie", 
    title="Powertrain repartition in the world",  
    data_reference=data_pie, 
    category_reference=categories_pie, 
    shape = 5,
    legend_position="b", 
    chart_position="D38", 
    ws=ws7, 
    width=17, 
    height=11,
    style=2
    
)

In [76]:
# Bubble chart kpi n°3 (can be improved in function)

chart3 = BubbleChart()

# Ajuster la taille du graphique
chart3.width = 17
chart3.height = 11

# mettre les données en référence
xvalues = Reference(ws6, min_col = 3,
                    min_row = 19, max_row = 25)
                     
yvalues = Reference(ws6, min_col = 4,
                    min_row = 19, max_row = 25)
                     
size = Reference(ws6, min_col = 5,
                min_row = 19, max_row = 25)

# créer une liste de valeurs
series = Series(values = yvalues, xvalues = xvalues,
                      zvalues = size, title ="EV stock")


chart3.series.append(series)

# les titres 
chart3.title = "Continent EV stock since 2011"
chart3.x_axis.title = "EV stock share 2023"
chart3.y_axis.title = " "
chart3.y_axis.delete = True

chart3.series[0].label = xvalues
chart3.dataLabels =DataLabelList()
chart3.dataLabels.showVal = True

# Positionner le graphique
ws7.add_chart(chart3, "AC38")
 
wb.save(str(filename))

In [77]:
# pie chart continent EV stock

# create data for plotting
categories_pie = Reference(ws6, min_col = 3, min_row = 19, max_row = 25) 
data_pie = Reference(ws6, min_col = 4, min_row = 18, max_row = 25) 

chart7 = create_chart(
    chart_type="pie", 
    title="EV stock global repartition",  
    data_reference=data_pie, 
    category_reference=categories_pie, 
    shape = 5,
    legend_position="b", 
    chart_position="AC13", 
    ws=ws7, 
    width=17, 
    height=11,
    style=19
    
)

In [78]:
# kpi n°4 Global evolution of EV sales linechart
chart4 = LineChart()
 
# créer le graph
values = Reference(ws6, min_col= 8, min_row=18, max_col = 9, max_row = 32) # PAS SÜR DE LA LIGNE DE DEPART
x_values = Reference(ws6, range_string="KPI!G19:G32")
 
#add data to the LineChart object
chart4.add_data(values, titles_from_data = True)
chart4.set_categories(x_values)
chart4.y_axis.scaling.logBase = 10 # avoir une échelle de y logarithmique car des valeurs très variées

# définir les titres
chart4.title = "Global evolution of EV sales"
chart4.x_axis.title = "Year"
chart4.y_axis.title = "Sales"

# Configurer les marqueurs pour chaque série de données
for i, series in enumerate(chart4.series):
    series.marker = Marker('circle')
    series.marker.size = 5
    series.graphicalProperties.line.noFill = False

# Ajuster la taille du graphique
chart4.width = 30
chart4.height = 11
chart4.style = 5

# Positionner la légende
chart4.legend.position = 'b'

# Positionner le graphique
ws7.add_chart(chart4, "N13")

# save the file
wb.save(str(filename))

In [79]:
# graph kpi n°6 : oil and electricity demand 
categories_bar = Reference(ws6, min_col=3, min_row=34, max_row=36)
data_bar = Reference(ws6, min_col=4, min_row=33, max_row=36, max_col=5)  

create_chart(
    chart_type="bar", 
    title="Oil displacement and Electricity demand", 
    x_title="Leading regions", 
    y_title="", 
    data_reference=data_bar, 
    category_reference=categories_bar, 
    shape = 4,
    legend_position="b", 
    chart_position="N38", 
    ws=ws7, 
    width=30, 
    height=11,
    style=10
    
)

#### Added information to dashboard

In [80]:
# Ajout d'information sur le graphique du kpi n°6
# A EFFACER ws7.merge_cells('R61:Z62')
r61 = ws7["R61"]

r61.value = "Oil displacement refers to the energy equivalent of the displaced oil in terms of 'millions of liters of gasoline'."

#ws7["R61"].alignment = center_aligned_text

wb.save(str(filename))

In [81]:
ws7['D61'].value = "Source : https://www.iea.org/data-and-statistics/data-tools/global-ev-data-explorer"
ws7.font = openpyxl.styles.Font(name = 'Arial', size = 11, color = 'FFFFFF', italic= True)
# A EFFACER ws7.merge_cells('D65:I65')

### Adapter la largeur des colonnes aux textes

In [82]:
# Adapter la largeur des colonnes
for j, col in enumerate(ws7.columns, 2):
   ws7.column_dimensions[openpyxl.utils.get_column_letter(j)].auto_size = True
for j, col in enumerate(ws6.columns, 2):
   ws6.column_dimensions[openpyxl.utils.get_column_letter(j)].auto_size = True

In [83]:
wb.save(str(filename))
wb.close()